# Mapas

En este notebook, vas a explorar distintas formas de generar mapas y visualizar datos geográficos en mapas interactivos con python.

## Librerías

`folium` es una librería que simplifica la generación de mapas interactivos en python.

In [1]:
import folium
from folium.plugins import DualMap

In [2]:
# permite que los gráficos se vean directamente en el notebook
%matplotlib inline 

# aplica una hoja de estilos para los gráficos
import matplotlib.pyplot as plt
plt.style.use('ggplot') 

In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon

## Mapas base

En forma similar a como funciona Google Maps, podés crear un mapa apuntando a alguna parte del planeta y modificando el zoom o el estilo del mapa base de fondo.

In [4]:
obelisco_coords = [-34.6037, -58.3815]

In [ ]:
mapa = folium.Map(location=obelisco_coords, zoom_start=17)

In [ ]:
mapa

`folium` te permite elegir entre algunos mapas base predefinidos, o usar un servicio como MapBox.

In [ ]:
folium.Map(location=obelisco_coords, zoom_start=17, tiles="Stamen Toner")

## Mapas de calor

Primero cargamos los datos que queremos mostrar en el mapa en un DataFrame.

In [ ]:
df_patentamientos = pd.read_csv("data/patentamientos_2018.csv", dtype={"provincia_id": str})

In [ ]:
df_patentamientos

,provincia_id,provincia_nombre,autos_patent_1000hab
0,14,Córdoba,21.956944
1,22,Chaco,16.265459
2,26,Chubut,27.813430
3,06,Buenos Aires,14.050967
4,10,Catamarca,17.638037
5,30,Entre Ríos,14.313720
6,34,Formosa,13.802050
7,42,La Pampa,24.981696
8,62,Río Negro,18.876913
9,70,San Juan,12.687321


Después armamos el mapa base. Como vamos a armar un mapa donde se ve toda la argentina, se puede usar la capa de OSM sin etiquetas o la capa del IGN con las etiquetas oficiales.

In [ ]:
# creamos mapa base sin etiquetas
argentina_coords = [-40, -64]
osm_no_labels = "https://tiles.wmflabs.org/osm-no-labels/{z}/{x}/{y}.png"
ign_labels = "http://wms.ign.gob.ar/geoserver/gwc/service/tms/1.0.0/capabaseargenmap@EPSG%3A3857@png/{z}/{x}/{y}.png"

mapa_patentamientos = folium.Map(location=argentina_coords, zoom_start=4)

folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa_patentamientos)

In [ ]:
mapa_patentamientos

Ahora tenemos que agregar la capa de provincias, sobre la cual vamos a graficar los datos que queramos. Podemos descargarla desde la API del Servicio de Normalización de Datos Geográficos de Argentina y luego simplificarla, porque la capa original es demasiado pesada para graficar en folium.

In [ ]:
# leemos usando GeoPandas
provincias = gpd.read_file("https://apis.datos.gob.ar/georef/api/provincias?formato=shp&max=100")
provincias.crs = {'init' :'epsg:3857'}

In [ ]:
# simplificamos cada una de las geometrías y las guardamos todas como MultiPolygon
tolerance = 0.1
provincias["geometry"] = [
    MultiPolygon([feature.simplify(tolerance=tolerance, preserve_topology=True)])
    if type(feature) == Polygon
    else feature.simplify(tolerance=tolerance, preserve_topology=True) 
    for feature in provincias["geometry"]
]

In [ ]:
# guardamos en un GeoJson, que es lo que folium necesitará para agregar la capa
provincias.to_file("data/provincias.geojson", driver='GeoJSON')

Ahora tenemos que agregar finalmente la capa de las provincias con los datos que queremos graficar en ella.

In [ ]:
folium.Choropleth(
    geo_data="data/provincias.geojson", # la capa de provincias en GeoJson
    data=df_patentamientos, # DataFrame con los datos a graficar
    columns=['provincia_id', 'autos_patent_1000hab'], # columna con id de geometrías, columna con el dato
    key_on='feature.properties.id', # campo del geojson que tiene el id de las geometrías
    fill_color='YlGn', # escala de colores a usar
    fill_opacity=0.7, # opacidad del color de relleno
    line_opacity=0.2, # opacidad de las líneas que separan los polígonos
    legend_name='Nuevas patentes cada 1000 habitantes', # título de la leyenda de la escala
).add_to(mapa_patentamientos)

In [ ]:
mapa_patentamientos

Tierra del Fuego se separa mucho del resto de las provincias, opacando el mapa de calor del resto por sus valores extremos. Para esto podemos especificar segmentos personalizados en el mapa de folium, por ejemplo usando cuantiles donde caiga la misma cantidad de provincias en cada uno.

In [ ]:
# creamos 5 bins con un quinto de las provincias en cada uno
bins = list(df_patentamientos['autos_patent_1000hab'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
bins

In [ ]:
# tenemos que resetear el mapa para cambiar la capa
mapa_patentamientos = folium.Map(location=argentina_coords, zoom_start=4)
folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa_patentamientos)

In [ ]:
folium.Choropleth(
    geo_data="data/provincias.geojson", # la capa de provincias en GeoJson
    data=df_patentamientos, # DataFrame con los datos a graficar
    columns=['provincia_id', 'autos_patent_1000hab'], # columna con id de geometrías, columna con el dato
    key_on='feature.properties.id', # campo del geojson que tiene el id de las geometrías
    fill_color='YlGn', # escala de colores a usar
    fill_opacity=0.7, # opacidad del color de relleno
    line_opacity=0.2, # opacidad de las líneas que separan los polígonos
    legend_name='Nuevas patentes cada 1000 habitantes', # título de la leyenda de la escala,
    bins=bins # especifica segmentos personalizados
).add_to(mapa_patentamientos)

In [ ]:
mapa_patentamientos

Ahora podemos querer agregar labels ubicados en los centroides de las provincias, para esto vamos a tener que crear algunas funciones.

In [ ]:
def add_label(folium_map, label, latitude, longitude, font_size=8):
    """Agrega un texto en determinadas coordenadas en un mapa de folium."""
    
    folium.map.Marker(
        [latitude, longitude],
        icon=folium.features.DivIcon(html='<div style="font-size: {}pt">{}</div>'.format(font_size, label))
    ).add_to(folium_map)

In [ ]:
def add_labels_from_column(folium_map, gdf, label_column, font_size=7):
    """Agrega textos de una columna de un GeoDataframe en centroides de las geometrías."""
    
    for centroid, label in zip(gdf.geometry.centroid, gdf[label_column]):
        latitude = centroid.coords[0][1]
        longitude = centroid.coords[0][0]
        add_label(folium_map, label, latitude, longitude, font_size=font_size)

In [ ]:
add_labels_from_column(mapa_patentamientos, provincias, "nombre")

mapa_patentamientos

Tal vez en lugar del nombre de las provincias queremos agregar el valor que estamos graficando, para esto hacemos un merge entre el geodataframe y el dataframe que contiene los valores graficados.

In [ ]:
provincias_merged = provincias.merge(df_patentamientos, left_on="id", right_on="provincia_id")

In [ ]:
# reseteamos el mapa
mapa_patentamientos = folium.Map(location=argentina_coords, zoom_start=4)
folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa_patentamientos)

# agregamos la capa con el mapa de calor
folium.Choropleth(
    geo_data="data/provincias.geojson", # la capa de provincias en GeoJson
    data=df_patentamientos, # DataFrame con los datos a graficar
    columns=['provincia_id', 'autos_patent_1000hab'], # columna con id de geometrías, columna con el dato
    key_on='feature.properties.id', # campo del geojson que tiene el id de las geometrías
    fill_color='YlGn', # escala de colores a usar
    fill_opacity=0.7, # opacidad del color de relleno
    line_opacity=0.2, # opacidad de las líneas que separan los polígonos
    legend_name='Nuevas patentes cada 1000 habitantes', # título de la leyenda de la escala,
    bins=bins # especifica segmentos personalizados
).add_to(mapa_patentamientos)

# agregamos los valores como labels
add_labels_from_column(mapa_patentamientos, provincias_merged, "autos_patent_1000hab")

mapa_patentamientos

El mapa anterior se ve mal porque el valor tiene demasiados decimales, esto se puede arreglar fácilmente redondeando los decimales de un dataframe con `df.round(1)`.

In [ ]:
# reseteamos el mapa
mapa_patentamientos = folium.Map(location=argentina_coords, zoom_start=4)
folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa_patentamientos)

# agregamos la capa con el mapa de calor
folium.Choropleth(
    geo_data="data/provincias.geojson", # la capa de provincias en GeoJson
    data=df_patentamientos, # DataFrame con los datos a graficar
    columns=['provincia_id', 'autos_patent_1000hab'], # columna con id de geometrías, columna con el dato
    key_on='feature.properties.id', # campo del geojson que tiene el id de las geometrías
    fill_color='YlGn', # escala de colores a usar
    fill_opacity=0.7, # opacidad del color de relleno
    line_opacity=0.2, # opacidad de las líneas que separan los polígonos
    legend_name='Nuevas patentes cada 1000 habitantes', # título de la leyenda de la escala,
    bins=bins # especifica segmentos personalizados
).add_to(mapa_patentamientos)

# agregamos los valores como labels
add_labels_from_column(mapa_patentamientos, provincias_merged.round(1), "autos_patent_1000hab")

mapa_patentamientos

Se puede crear también un mapa dual, donde podemos elegir asignar las capas a ambos mapas, a la izquierda o a la derecha.

In [ ]:
# creamos un mapa dual tal como si fuera un mapa normal
mapa_dual = folium.plugins.DualMap(location=argentina_coords, zoom_start=4)
folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa_dual)

mapa_dual

In [ ]:
# creamos un mapa dual tal como si fuera un mapa normal
mapa_dual = folium.plugins.DualMap(location=argentina_coords, zoom_start=4)
folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa_dual)

# agregamos la capa con el mapa de calor SIN bins personalizados al mapa de la izquierda
folium.Choropleth(
    geo_data="data/provincias.geojson", # la capa de provincias en GeoJson
    data=df_patentamientos, # DataFrame con los datos a graficar
    columns=['provincia_id', 'autos_patent_1000hab'], # columna con id de geometrías, columna con el dato
    key_on='feature.properties.id', # campo del geojson que tiene el id de las geometrías
    fill_color='YlGn', # escala de colores a usar
    fill_opacity=0.7, # opacidad del color de relleno
    line_opacity=0.2, # opacidad de las líneas que separan los polígonos
    legend_name='Nuevas patentes cada 1000 habitantes' # título de la leyenda de la escala,
).add_to(mapa_dual)

# agregamos los valores como labels
add_labels_from_column(mapa_dual.m1, provincias_merged.round(1), "autos_patent_1000hab")
add_labels_from_column(mapa_dual.m2, provincias_merged, "nombre")

mapa_dual